<a href="https://colab.research.google.com/github/lauxpaux/bank_small_dataset/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlib inline

In [23]:
url = 'https://raw.githubusercontent.com/lauxpaux/bank_small_dataset/main/bank_small.csv'
df = pd.read_csv(url)
df

,loan,duration,poutcome,job,y
0,no,1042,unknown,admin.,yes
1,no,1467,unknown,admin.,yes
2,no,1389,unknown,technician,yes
3,no,579,unknown,services,yes
4,no,673,unknown,admin.,yes
...,...,...,...,...,...
15284,no,171,unknown,management,no
15285,no,91,unknown,management,no
15286,no,608,unknown,blue-collar,no
15287,no,100,unknown,entrepreneur,no


### First glance: Data cleaning and data processing

* Check null values and unique values for loan
* convert duration from seconds to other timestamp values (minutes, date?)
* poutcome - check unique values, maybe do one hot encoding if value is other than unknown
* job - check different job options and one hot encode values
* y - check for null values, plot the values then drop column


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15289 entries, 0 to 15288
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   loan      15289 non-null  object
 1   duration  15289 non-null  int64 
 2   poutcome  15289 non-null  object
 3   job       15289 non-null  object
 4   y         15289 non-null  object
dtypes: int64(1), object(4)
memory usage: 597.4+ KB


In [21]:
df.describe()

,duration
count,15289.000000
mean,330.878671
std,322.269855
min,0.000000
25%,124.000000
50%,222.000000
75%,417.000000
max,3881.000000


In [24]:
null_vals = df.loc[df.isnull().any(axis=1)]
null_vals

,loan,duration,poutcome,job,y


Let's look at the unique values represented in each column. This will help us determine the options that we are working with and it helps us better gauge what our final one-hot-encoded dataset will look like.

In [32]:
cols = ['loan','poutcome', 'job', 'y']
for col in df:
  print(df[col].unique())


['no' 'yes']
[1042 1467 1389 ...  948 3322 2027]
['unknown' 'other' 'failure' 'success']
['admin.' 'technician' 'services' 'management' 'retired' 'unemployed'
 'blue-collar' 'entrepreneur' 'housemaid' 'self-employed' 'unknown'
 'student']
['yes' 'no']
